In [1]:
from types import SimpleNamespace
import numpy as np

## algorithm

In [2]:
class HashTable:

    ratio_expand = .8
    ratio_shrink = .2
    min_size = 11

    def __init__(self, size=None):
        self._size = size or self.min_size
        self._buckets = [None] * self._size
        self._list = None
        self._count = 0

    def _entry(self, key):
        # get hash and index
        idx = hash(key) % self._size

        # find entry by key
        p, q = self._buckets[idx], None
        while p and p.key != key:
            p, q = p.next, p

        # index, entry, previous entry
        return idx, p, q

    def _ensure_capacity(self):
        fill = self._count / self._size
        
        # expand or shrink?
        if fill > self.ratio_expand:
            self._size = self._size * 2 + 1
        elif fill < self.ratio_shrink and self._size > self.min_size:
            self._size = (self._size - 1) // 2
        else:
            return

        # reallocate buckets
        self._buckets = [None] * self._size

        # store entries into new buckets
        p = self._list
        while p:
            idx = hash(p.key) % self._size
            p.next = self._buckets[idx]
            self._buckets[idx] = p
            p = p.entry_next

    def __len__(self):
        return self._count

    def __contains__(self, key):
        _, p, _ = self._entry(key)
        return bool(p)

    def __getitem__(self, key):
        _, p, _ = self._entry(key)
        return p and p.value

    def __setitem__(self, key, value):
        idx, p, _ = self._entry(key)

        # set entry if key was found
        if p:
            p.value = value
            return

        # create new entry
        p = SimpleNamespace(
            key=key,
            value=value,
            next=self._buckets[idx],
            entry_next=self._list,
            entry_prev=None
        )

        # store to bucket
        self._buckets[idx] = p

        # store to list
        if self._list:
            self._list.entry_prev = p
        self._list = p

        # expand
        self._count += 1
        self._ensure_capacity()

    def __delitem__(self, key):
        idx, p, q = self._entry(key)

        # key not found
        if not p:
            return

        # remove from bucket
        if q:
            q.next = p.next
        else:
            self._buckets[idx] = p.next

        # remove from list
        if p.entry_next:
            p.entry_next.entry_prev = p.entry_prev
        if p.entry_prev:
            p.entry_prev.entry_next = p.entry_next
        else:
            self._list = p.entry_next

        # shrink
        self._count -= 1
        self._ensure_capacity()

    def __iter__(self):
        p = self._list
        while p:
            yield p.key
            p = p.entry_next
            
    def slots(self):
        return ''.join(p and 'x' or '-' for p in self._buckets)

## run

In [3]:
table = HashTable()

In [4]:
# add random values
for _ in range(1000):
    key, value = np.random.randint(1000), np.random.rand()
    table[key] = value

In [5]:
len(table), table._size

(625, 1535)

In [6]:
table.slots()

'-x-x-x-xx-xxxxx-xxx-xxxx-x-x-x---xx-xxx---xx-x--x-xx-x-xxxx-xx-xx--xxx--xx-x-x-xx-x-xxxx-x-xxxx--xxxxx-x--xxx-xxx-xxxxxx--xxxx-xxxxx---xxxxxx--xxxxxxx--xx-x-xx--xx-xxxxx-x-xxxxx-xx--x--xxx-xxxx-x-xx-x---x-xx---xx--xxxxx--xx-x---x--xx-----xxx--x--xx-xx--xxxx-xx---xxxx-x-xxx--x-xxx--xx-xx-xx-xx--xx-xx-xxxxxx-xxxxxx--xx-x-xx---xx--x-xxx--xx-xx-x-xx-x-x-xxxx--xx-xx-x-xx-x--xxx-xx---xx--xx--x-xxx---x-xxxxxxx--xxx-x---x-x-x--x-xxxxx---x-xxxxx----xx-xxxx-x-xxxxxxxxx-xxxxxx---xx-x--xx-xx-xxx--xxx-x---xxxxx--x-xx-xx---xxx-xx--xx-xx-xx-xx-xxxxx--xx-xxx-x----x-xxxx--xxxx-xxxxxx--xxxxxx-x-x-x--x-xxx--xx-x-x-x-xxxx-xx-x-x-x-xxx--xxxxxx-xxxxxxx-x-x-xxxxxxx-xxx-xxx-x-xxx-xxxxx---x-xxxx-x-x-x--x---xxx--xxxx-xxxx----xxxx-xxx-xxx-xx-x-x-xxx-x---xxx-xx-x--x-xx--xxx-x-xxxxxx-xxx--xx----x-xxxxxxx-------x-x-xxxxxxx-x-x-x-----xxxxx-xxxxx--xxx-xxx-----xxx-xxxx--xxxxxxx--xx-xxx-xx---xx-xxxxxxxxxxxxxx-xxxx--x--x--xxxx---xxxxxx-xxxx---xxx---x-xxxxxx-x--xxxx--xxxxx---xxx-x-x---xxx---xx-xx-x-xxxx--xxx-xxxxxx--xx-

In [7]:
# print some values
for key in list(table)[:5]:
    print(key, table[key])

918 0.46964946620099624
880 0.6586348500634474
633 0.06134674099379567
89 0.7216673947800543
53 0.7986314320745663


In [8]:
# delete all the values
for key in list(table):
    del table[key]

In [9]:
len(table), table._size

(0, 11)